In [14]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score

In [3]:
device = torch.device('cpu')
df_test_text = pd.read_csv("test.csv")
df_test_labels = pd.read_csv("test_labels.csv")
df_test = pd.merge(df_test_text, df_test_labels, on='id')
df_test.head(10)

df_test['text'] = df_test['comment_text']
df_test.drop(['id', 'comment_text'], inplace=True, axis=1)
df_test.head(20)

df_test = df_test.drop(df_test[df_test['toxic'] == -1].index)
df_test

,toxic,severe_toxic,obscene,threat,insult,identity_hate,text
5,0,0,0,0,0,0,Thank you for understanding. I think very high...
7,0,0,0,0,0,0,:Dear god this site is horrible.
11,0,0,0,0,0,0,"""::: Somebody will invariably try to add Relig..."
13,0,0,0,0,0,0,""" \n\n It says it right there that it IS a typ..."
14,0,0,0,0,0,0,""" \n\n == Before adding a new product to the l..."
...,...,...,...,...,...,...,...
153150,0,0,0,0,0,0,":Jerome, I see you never got around to this…! ..."
153151,0,0,0,0,0,0,==Lucky bastard== \n http://wikimediafoundatio...
153154,0,0,0,0,0,0,==shame on you all!!!== \n\n You want to speak...
153155,1,0,1,0,1,0,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...


In [5]:
def get_pred(text, classifier, tokenizer):
    text_token = tokenizer("hello world", return_tensors="pt")
    output = classifier(**text_token)
    prediction = torch.sigmoid(output.logits)*100
    prediction = prediction.detach().numpy().tolist()[0]
    return prediction

In [ ]:
y_pred = []
y_true = []
classifier = AutoModelForSequenceClassification.from_pretrained("sachiniyer/tweet_toxicity")
tokenizer = AutoTokenizer.from_pretrained("sachiniyer/tweet_toxicity")
for _, d in tqdm(enumerate(df_test.itertuples(), 0)):
    y_pred.append([round(i) for i in get_pred(d.text, classifier, tokenizer)])
    y_true.append([d.toxic, d.severe_toxic, d.obscene, d.threat, d.insult, d.identity_hate])

62295it [09:33, 108.41it/s]

In [25]:
test_y_pred = y_pred[0:10]
test_y_true = y_true[0:10]
test_y_true

[Pandas(Index=5, toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0, text='Thank you for understanding. I think very highly of you and would not revert without discussion.'),
 Pandas(Index=7, toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0, text=':Dear god this site is horrible.'),
 Pandas(Index=11, toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0, text='"::: Somebody will invariably try to add Religion?  Really??  You mean, the way people have invariably kept adding ""Religion"" to the Samuel Beckett infobox?  And why do you bother bringing up the long-dead completely non-existent ""Influences"" issue?  You\'re just flailing, making up crap on the fly. \n ::: For comparison, the only explicit acknowledgement in the entire Amos Oz article that he is personally Jewish is in the categories!    \n\n "'),
 Pandas(Index=13, toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0, text='" \n\n It says it righ

In [17]:
accuracy_score(y_pred, y_true)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass-multioutput targets